In [1]:
from sklearn import datasets
from scipy.io.arff import loadarff
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd


In [2]:
forest = datasets.fetch_covtype()

In [3]:
type(forest)

sklearn.datasets.base.Bunch

In [18]:
X, X_test, y, y_test = train_test_split(
    forest.data, forest.target == 2, test_size=0.3, random_state=1234)

In [19]:
print(X.shape, y.shape, X_test.shape, y_test.shape)

(406708, 54) (406708,) (174304, 54) (174304,)


## Maintenant un peu de xgboost!

### D'abord un tout petit

In [11]:
import numpy as np
def logit(p):
    return np.log(p) - np.log(1 - p)

def inv_logit(p):
    return np.exp(p) / (1 + np.exp(p))

In [8]:
import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc'
}

num_round = 4
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.81286	train-auc:0.812862


In [9]:
X_test[0, [0,25,35,15]]

array([ 2687.,     0.,     0.,     0.])

In [12]:
leaves = bst.predict(dtest, pred_leaf=True)
z = bst.predict(dtest)
leaves[:3], z[:3], inv_logit(0.388039+ 0.354413)

(array([[77, 73, 81, 82],
        [77, 73, 83, 80],
        [77, 73, 81, 82]], dtype=int32),
 array([ 0.73306412,  0.59011638,  0.73306412], dtype=float32),
 0.67753180825889836)

In [13]:
leaves[400], y[400], X_test[400, [0,7,3]]

(array([101, 104,  99,  89], dtype=int32),
 0.51603085,
 array([ 3126.,   254.,   283.]))

In [14]:
bst.dump_model("bst_2_trees")
! cat bst_2_trees

booster[0]:
0:[f0<3073.5] yes=1,no=2,missing=1
	1:[f0<2497.5] yes=3,no=4,missing=3
		3:[f0<2436.5] yes=7,no=8,missing=7
			7:[f5<659.5] yes=15,no=16,missing=15
				15:[f0<2220.5] yes=31,no=32,missing=31
					31:[f0<2165.5] yes=63,no=64,missing=63
						63:leaf=-0.597907
						64:leaf=-0.55443
					32:[f23<0.5] yes=65,no=66,missing=65
						65:leaf=-0.556008
						66:leaf=-0.387296
				16:[f9<1375] yes=33,no=34,missing=33
					33:leaf=-0.597343
					34:[f0<2379.5] yes=67,no=68,missing=67
						67:leaf=-0.595745
						68:leaf=-0.446747
			8:[f23<0.5] yes=17,no=18,missing=17
				17:[f10<0.5] yes=35,no=36,missing=35
					35:[f44<0.5] yes=69,no=70,missing=69
						69:leaf=-0.512204
						70:leaf=0.538983
					36:[f9<5104.5] yes=71,no=72,missing=71
						71:leaf=0.542857
						72:leaf=-0.544186
				18:[f3<205.5] yes=37,no=38,missing=37
					37:[f5<1357.5] yes=73,no=74,missing=73
						73:leaf=-0.0372881
						74:leaf=-0.565517
					38:[f12<0.5] yes=75,no=76,missing=75
						75:leaf=-0.537162

In [21]:
X.shape, y.shape, X_test.shape, y_test.shape

((406708, 54), (406708,), (174304, 54), (174304,))

In [22]:
import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc'
}

num_round = 201
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.81286	train-auc:0.812862
[40]	eval-auc:0.920766	train-auc:0.924006
[80]	eval-auc:0.940982	train-auc:0.945078
[120]	eval-auc:0.956681	train-auc:0.961603
[160]	eval-auc:0.964821	train-auc:0.970127
[200]	eval-auc:0.970554	train-auc:0.976311


In [25]:
param = {
    'bst:max_depth': 7, 
    'bst:eta': 0.2,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc'
}

num_round = 401
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.813996	train-auc:0.814956
[40]	eval-auc:0.916638	train-auc:0.919448
[80]	eval-auc:0.936904	train-auc:0.940796
[120]	eval-auc:0.949872	train-auc:0.954719
[160]	eval-auc:0.958275	train-auc:0.963817
[200]	eval-auc:0.964482	train-auc:0.970541
[240]	eval-auc:0.969145	train-auc:0.975591
[280]	eval-auc:0.972509	train-auc:0.97925
[320]	eval-auc:0.97558	train-auc:0.982673
[360]	eval-auc:0.978116	train-auc:0.98538
[400]	eval-auc:0.979837	train-auc:0.987093
